imports

In [46]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import torch
import math

from myhelpers import config_plots
config_plots.global_settings()

parameters

In [47]:
species_csv_fileName = "TableS2HoyalCuthilletal2019ScienceAdvances.csv"
species_csv_fileName_cleaned = "TableS2HoyalCuthilletal2019ScienceAdvances_cleaned.csv"

image_subpath = "images"
species_csv_fileName_header = "Image filename"
species_csv_scientificName_header = "Species"
species_csv_fullsubspecies_header= "Fullsubspecies"
species_csv_view_header = "View"
species_csv_subspecies_header = "Subspecies"
species_csv_sex_header = "Sex"

species_csv_usedColumns = [species_csv_fileName_header,
                          species_csv_scientificName_header,
                          species_csv_view_header,
                          species_csv_subspecies_header,
                          species_csv_sex_header]

dataPath="/raid/elhamod/Butterflies2/Datasets/LowResolution"

suffix="full"

cuda=1

numberOfImagesPerSpecies_min =4 # If None, no min
numberOfImagesPerSpecies_toKeep =None # If None, keep all

In [ ]:
# set cuda
if torch.cuda.is_available():
    torch.cuda.set_device(cuda)
    print("using cuda", cuda)

Subset creation

In [53]:
dir_path = os.path.join(dataPath, suffix)
file_path = os.path.join(dir_path, species_csv_fileName)
if not os.path.exists(file_path):
    print(file_path, "File not found")
    raise

# Read data frame
df = pd.read_csv(file_path, delimiter=',', index_col=species_csv_fileName_header, usecols=species_csv_usedColumns)
df[species_csv_fullsubspecies_header] = df[species_csv_scientificName_header] + " " + df[species_csv_subspecies_header]

# find species that have a specific number of images
grouped_by = df[species_csv_fullsubspecies_header].value_counts()
if numberOfImagesPerSpecies_min is not None:
    grouped_by = grouped_by[grouped_by > numberOfImagesPerSpecies_min]
scientificNameValues = grouped_by.index.values
thresholded = df[df[species_csv_fullsubspecies_header].isin(scientificNameValues)]

# Get first n images from each
if numberOfImagesPerSpecies_toKeep is not None:
    trimmed = thresholded.groupby(species_csv_fullsubspecies_header, group_keys=False)
    trimmed = trimmed.apply(lambda x: x.sample(n=numberOfImagesPerSpecies_toKeep) if len(x) >= numberOfImagesPerSpecies_toKeep else x)
    trimmed.reset_index(drop=True)
else:
    trimmed = thresholded
print('output', trimmed)

# save the file
path = os.path.join(dataPath, suffix)
trimmed.to_csv(os.path.join(path, species_csv_fileName_cleaned), sep=',')

output                    fileName         scientificName        Genus         Family
0      INHS_FISH_103994.jpg       Gambusia affinis     Gambusia    Poeciliidae
1      INHS_FISH_108425.jpg    Notropis stramineus     Notropis     Cyprinidae
2       INHS_FISH_59772.jpg      Lepomis cyanellus      Lepomis  Centrarchidae
3        INHS_FISH_4051.jpg        Noturus gyrinus      Noturus    Ictaluridae
4       INHS_FISH_21948.jpg  Notropis atherinoides     Notropis     Cyprinidae
...                     ...                    ...          ...            ...
2595    INHS_FISH_31114.jpg       Notropis greenei     Notropis     Cyprinidae
2596    INHS_FISH_75551.jpg     Lepomis marginatus      Lepomis  Centrarchidae
2597  INHS_FISH_52739_2.jpg   Lepisosteus oculatus  Lepisosteus  Lepisosteidae
2598    INHS_FISH_81504.jpg       Notropis greenei     Notropis     Cyprinidae
2599    INHS_FISH_85916.jpg   Lepisosteus oculatus  Lepisosteus  Lepisosteidae

[2600 rows x 4 columns]
